# [심층신경망개론] Group 1 ConvNeXt 구현
- [ConvNeXt V2: Co-designing and Scaling ConvNets with Masked Autoencoders](https://arxiv.org/pdf/2301.00808)
- [GitHub](https://github.com/facebookresearch/ConvNeXt-V2)

## Data Preprocessing
- ConvNeXt Input 형식에 맞게 [224x224] 크기의 이미지 생성
- ConvNeXt Input channel은 RGB로 3임. (모델 또는 Dataset 수정)
- PTB-XL 데이터셋은 ECG 파형뿐만 아니라, 환자 정보를 담고 있음. (Multi-modal)
    - [An Improved ConvNeXt with Multimodal Transformer for Physiological Signal Classification](https://vuir.vu.edu.au/48410/1/An_Improved_ConvNeXt_with_Multimodal_Transformer_for_Physiological_Signal_Classification.pdf)

In [ ]:
import pandas as pd
import numpy as np
import wfdb
import ast
import os
from sklearn.preprocessing import MultiLabelBinarizer

def load_raw_data(df, sampling_rate, path):
    """df.index를 기준으로 데이터를 로드"""
    if sampling_rate == 100:
        data = [wfdb.rdsamp(os.path.join(path, f)) for f in df['filename_lr']]
    else:
        data = [wfdb.rdsamp(os.path.join(path, f)) for f in df['filename_hr']]
    data = np.array([signal for signal, meta in data])
    return data

# 데이터 경로 설정
path = r"C:\Users\은혁\Desktop\심신개 Project\Data\ptbxl"
sampling_rate = 100

# PTB-XL 데이터베이스 로드
df = pd.read_csv(os.path.join(path, 'ptbxl_database.csv'), index_col='ecg_id')
df.scp_codes = df.scp_codes.apply(lambda x: ast.literal_eval(x))

# 진단 정보 로드
agg_df = pd.read_csv(os.path.join(path, 'scp_statements.csv'), index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    """진단 클래스를 매핑하는 함수"""
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

df['diagnostic_superclass'] = df.scp_codes.apply(aggregate_diagnostic)

df = df[df['diagnostic_superclass'].apply(lambda x: len(x) > 0)]

# Raw data 로드
X = load_raw_data(df, sampling_rate, path)
assert len(X) == len(df), "X와 df의 크기가 일치하지 않습니다."

# 데이터셋 분리
test_fold = 10
val_fold = 9

train_filter = (df.strat_fold != test_fold) & (df.strat_fold != val_fold)
val_filter = df.strat_fold == val_fold
test_filter = df.strat_fold == test_fold

X_train = X[train_filter]
y_train = list(df[train_filter]['diagnostic_superclass'])

X_val = X[val_filter]
y_val = list(df[val_filter]['diagnostic_superclass'])

X_test = X[test_filter]
y_test = list(df[test_filter]['diagnostic_superclass'])

mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_val_bin = mlb.transform(y_val)
y_test_bin = mlb.transform(y_test)

print(f"Train Data Shape: {X_train.shape}, Labels: {y_train_bin.shape}")
print(f"Validation Data Shape: {X_val.shape}, Labels: {y_val_bin.shape}")
print(f"Test Data Shape: {X_test.shape}, Labels: {y_test_bin.shape}")

## Data Transform

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, DeiTForImageClassification
from torchvision.models import convnext_tiny, efficientnet_v2_s
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Configurations ---
args = {
    "batch_size": 64,
    "epochs": 40,
    "model": "convnextv2_tiny",
    "input_size": 224,
    "lr": 4e-3,
    "weight_decay": 0.05,
    # "warmup_epochs": 20,
    # "mixup": 0.8,
    # "cutmix": 1.0,
    # "smoothing": 0.1,
    "data_path": path,
    "output_dir": r"C:\Users\은혁\Desktop\심신개 Project\checkpoints\convnextv2_tiny",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "num_workers": 0,
    "use_amp": False,
}

# --- Data Preparation ---
class ECGDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample.float(), torch.tensor(label, dtype=torch.int64)

transform = transforms.Compose([
    transforms.ToTensor(),  # Numpy 배열 -> Tensor
    transforms.Resize((182, 256)),  # 이미지 크기 조정
    transforms.Lambda(lambda x: x.expand(3, -1, -1)),  # 1채널 데이터를 3채널로 확장
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    # transforms.RandomResizedCrop(args["input_size"]),
    # transforms.RandomHorizontalFlip(),
    transforms.Lambda(lambda x: x.expand(3, -1, -1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# --- Data Loader ---
train_dataset = ECGDataset(X_train, y_train_bin, transform=transform)
valid_dataset = ECGDataset(X_val, y_val_bin, transform=transform)
test_dataset = ECGDataset(X_test, y_test_bin, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

class_names = mlb.classes_
print(f"\nClasses: {class_names}")

print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(valid_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")


Classes: ['CD' 'HYP' 'MI' 'NORM' 'STTC']
Train dataset length: 17084
Validation dataset length: 2146
Test dataset length: 2158


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
next(iter(train_loader))

[tensor([[[[-0.9919, -0.9919, -0.9919,  ..., -0.7681, -0.7681, -0.7681],
           [-0.9504, -0.9504, -0.9504,  ..., -0.7193, -0.7193, -0.7193],
           [-0.9964, -0.9964, -0.9964,  ..., -0.8101, -0.8101, -0.8101],
           ...,
           [-1.2031, -1.2031, -1.2031,  ..., -0.9059, -0.9059, -0.9059],
           [-1.1189, -1.1189, -1.1189,  ..., -0.8946, -0.8946, -0.8946],
           [-1.0773, -1.0773, -1.0773,  ..., -0.8185, -0.8185, -0.8185]],
 
          [[-0.9919, -0.9919, -0.9919,  ..., -0.7681, -0.7681, -0.7681],
           [-0.9504, -0.9504, -0.9504,  ..., -0.7193, -0.7193, -0.7193],
           [-0.9964, -0.9964, -0.9964,  ..., -0.8101, -0.8101, -0.8101],
           ...,
           [-1.2031, -1.2031, -1.2031,  ..., -0.9059, -0.9059, -0.9059],
           [-1.1189, -1.1189, -1.1189,  ..., -0.8946, -0.8946, -0.8946],
           [-1.0773, -1.0773, -1.0773,  ..., -0.8185, -0.8185, -0.8185]],
 
          [[-0.9919, -0.9919, -0.9919,  ..., -0.7681, -0.7681, -0.7681],
           [-

## Models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath

class LayerNorm(nn.Module):
    """ LayerNorm that supports two data formats: channels_last (default) or channels_first. 
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with 
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs 
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x

class GRN(nn.Module):
    """ GRN (Global Response Normalization) layer
    """
    def __init__(self, dim):
        super().__init__()
        self.gamma = nn.Parameter(torch.zeros(1, 1, 1, dim))
        self.beta = nn.Parameter(torch.zeros(1, 1, 1, dim))

    def forward(self, x):
        Gx = torch.norm(x, p=2, dim=(1,2), keepdim=True)
        Nx = Gx / (Gx.mean(dim=-1, keepdim=True) + 1e-6)
        return self.gamma * (x * Nx) + self.beta + x

class Block(nn.Module):
    """ ConvNeXtV2 Block.
    
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
    """
    def __init__(self, dim, drop_path=0.):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.grn = GRN(4 * dim)
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.grn(x)
        x = self.pwconv2(x)
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

class ConvNeXtV2(nn.Module):
    """ ConvNeXt V2
        
    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, num_classes=1000, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], 
                 drop_path_rate=0., head_init_scale=1.
                 ):
        super().__init__()
        self.depths = depths
        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j]) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6) # final norm layer
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1])) # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

def convnextv2_atto(**kwargs):
    model = ConvNeXtV2(depths=[2, 2, 6, 2], dims=[40, 80, 160, 320], **kwargs)
    return model

def convnextv2_femto(**kwargs):
    model = ConvNeXtV2(depths=[2, 2, 6, 2], dims=[48, 96, 192, 384], **kwargs)
    return model

def convnext_pico(**kwargs):
    model = ConvNeXtV2(depths=[2, 2, 6, 2], dims=[64, 128, 256, 512], **kwargs)
    return model

def convnextv2_nano(**kwargs):
    model = ConvNeXtV2(depths=[2, 2, 8, 2], dims=[80, 160, 320, 640], **kwargs)
    return model

def convnextv2_tiny(**kwargs):
    model = ConvNeXtV2(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    return model

def convnextv2_base(**kwargs):
    model = ConvNeXtV2(depths=[3, 3, 27, 3], dims=[128, 256, 512, 1024], **kwargs)
    return model

def convnextv2_large(**kwargs):
    model = ConvNeXtV2(depths=[3, 3, 27, 3], dims=[192, 384, 768, 1536], **kwargs)
    return model

def convnextv2_huge(**kwargs):
    model = ConvNeXtV2(depths=[3, 3, 27, 3], dims=[352, 704, 1408, 2816], **kwargs)
    return model

c:\anaconda3\envs\vingt\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\anaconda3\envs\vingt\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
C:\Users\은혁\AppData\Local\Temp\ipykernel_26944\2224499961.py:151: UserWarning: Overwriting convnext_tiny in registry with __main__.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(pretrained=False,in_22k=False, **kwargs):
C:\Users\은혁\AppData\Local\Temp\ipykernel_26944\2224499961.py:163: UserWarning: Overwriting convnext_small in registry with __main__

In [ ]:
from timm import create_model
from timm.loss import SoftTargetCrossEntropy

# --- Model Setup ---
model = create_model(args["model"], pretrained=False, num_classes=y_train_bin.shape[1])
model = model.to(args["device"])

criterion = SoftTargetCrossEntropy()
#criterion = LabelSmoothingCrossEntropy(smoothing=args["smoothing"])
optimizer = optim.AdamW(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args["epochs"])

print(model)

ConvNeXt(
  (downsample_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm()
    )
    (1): Sequential(
      (0): LayerNorm()
      (1): Conv2d(96, 192, kernel_size=(2, 2), stride=(2, 2))
    )
    (2): Sequential(
      (0): LayerNorm()
      (1): Conv2d(192, 384, kernel_size=(2, 2), stride=(2, 2))
    )
    (3): Sequential(
      (0): LayerNorm()
      (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
    )
  )
  (stages): ModuleList(
    (0): Sequential(
      (0): Block(
        (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
        (norm): LayerNorm()
        (pwconv1): Linear(in_features=96, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (pwconv2): Linear(in_features=384, out_features=96, bias=True)
        (drop_path): Identity()
      )
      (1): Block(
        (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(

## Train

In [ ]:
import os
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

save_dir = args["output_dir"]
os.makedirs(save_dir, exist_ok=True)

# --- Training Loop ---
scaler = torch.amp.GradScaler(enabled=args["use_amp"])
for epoch in range(args["epochs"]):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{args['epochs']}", ncols=100):
        images, labels = images.to(args["device"]), labels.to(args["device"])

        """
        if mixup_fn is not None:
            images, labels = mixup_fn(images, labels)
        """
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        all_preds.append(torch.argmax(outputs, dim=1).cpu().numpy())
        all_labels.append(torch.argmax(labels, dim=1).cpu().numpy())

    scheduler.step()
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    train_accuracy = (all_preds == all_labels).mean()
    train_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    train_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    train_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    print(f"Epoch [{epoch+1}/{args['epochs']}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Accuracy: {train_accuracy:.4f}, F1 Score: {train_f1:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")
    
    save_path = os.path.join(save_dir, f"ConvNeXtV2_epoch_{epoch+1}.pth")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved at {save_path}")

    # --- Validation Loop ---
    model.eval()
    valid_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(valid_loader, desc=f"Validating Epoch {epoch+1}/{args['epochs']}", ncols=100):
            images, labels = images.to(args["device"]), labels.to(args["device"])
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            all_preds.append(torch.argmax(outputs, dim=1).cpu().numpy())
            all_labels.append(torch.argmax(labels, dim=1).cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    val_accuracy = (all_preds == all_labels).mean()
    val_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    val_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    val_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    print(f"Validation Loss: {valid_loss/len(valid_loader):.4f}, Accuracy: {val_accuracy:.4f}, "
          f"F1 Score: {val_f1:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}\n")

Using device: cuda


Training Epoch 1/40: 100%|██████████████████████████████████████| 1068/1068 [05:49<00:00,  3.05it/s]


Epoch [1/40], Loss: 2.0653, Accuracy: 0.4042, F1 Score: 0.2672, Precision: 0.2743, Recall: 0.4042
Model saved at C:\Users\은혁\Desktop\심신개 Project\checkpoints\ConvNeXt_epoch_1.pth


Validating Epoch 1/40: 100%|██████████████████████████████████████| 135/135 [00:14<00:00,  9.55it/s]


Validation Loss: 1.9858, Accuracy: 0.4278, F1 Score: 0.2563, Precision: 0.1830, Recall: 0.4278



...

Validation Loss: 1.9836, Accuracy: 0.4278, F1 Score: 0.2563, Precision: 0.1830, Recall: 0.4278



Training Epoch 40/40: 100%|█████████████████████████████████████| 1068/1068 [05:21<00:00,  3.32it/s]


Epoch [40/40], Loss: 1.9897, Accuracy: 0.4251, F1 Score: 0.2536, Precision: 0.1807, Recall: 0.4251
Model saved at C:\Users\은혁\Desktop\심신개 Project\checkpoints\ConvNeXt_epoch_40.pth


Validating Epoch 40/40: 100%|█████████████████████████████████████| 135/135 [00:13<00:00, 10.22it/s]

Validation Loss: 1.9836, Accuracy: 0.4278, F1 Score: 0.2563, Precision: 0.1830, Recall: 0.4278



## Test

In [ ]:
# --- Test Loop ---
model.eval()
test_loss = 0.0
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(args["device"]), labels.to(args["device"])
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        all_preds.append(torch.argmax(outputs, dim=1).cpu().numpy())
        all_labels.append(torch.argmax(labels, dim=1).cpu().numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

test_accuracy = (all_preds == all_labels).mean()
test_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
test_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
test_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"Test Loss: {test_loss/len(test_loader):.4f}, Accuracy: {test_accuracy:.4f}, "
      f"F1 Score: {test_f1:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")

Test Loss: 1.9795, Accuracy: 0.4245, F1 Score: 0.2530, Precision: 0.1802, Recall: 0.4245
